## Morphological reconstruction

In [1]:
import cv2
import numpy as np

def morph_recons(resized_img,width,height):
    
    kernel = np.ones((8,8),np.uint8)
    erosion = cv2.erode(resized_img,kernel,iterations = 1)

    return erosion
 




# l0 gradient minimization

In [2]:
import numpy as np
from scipy.fftpack import fft, ifft, fft2, ifft2, fftshift, ifftshift
import matplotlib.pyplot as plt

import sys
import os
import argparse
from skimage import io
import skimage.data
import skimage.transform
import skimage.color
from PIL import Image
import scipy.misc
import matplotlib

In [3]:

def get_configuration(filename,add_arguments_func=None):
    lmd=0.02
    beta_max=1.0e5
    beta_rate=2.0
    resized=450

    img = io.imread(filename)

    # resize
    size = resized
    print(f"img.shape:{img.shape}")
    org_h, org_w = img.shape[:2]
    if org_h < org_w:
        w, h = (size, size*org_h/org_w)
    else:
        w, h = (size*org_w/org_h, size)
    
    w =600
    h = 450
    print(f"{org_w}, {org_h}, {w}, {h}")
    print('{}: {}x{} -> {}x{}'.format(filename, org_w, org_h, w, h))
    img = skimage.transform.resize(img, (int(h), int(w)))

    # access to commonly used arguments
    return img, (lmd, beta_max, beta_rate)



In [4]:

def clip_img(img):
    return np.clip(img, 0, 1)
def add_noise(img, sigma):
    return  clip_img(img + np.random.randn(*img.shape) * sigma)

In [5]:

def l0_gradient_minimization_2d(I, lmd, beta_max, beta_rate=2.0, max_iter=30, return_history=False):
    u'''image I can be both 1ch (ndim=2) or D-ch (ndim=D)'''
    S = np.array(I)

    # prepare FFT
    F_I = fft2(S, axes=(0, 1))
    Ny, Nx = S.shape[:2]
    D = S.shape[2] if S.ndim == 3 else 1
    dx, dy = np.zeros((Ny, Nx)), np.zeros((Ny, Nx))
    print(f"{Ny/2}, {Nx/2-1},{Nx/2+1}")
    dx[int(Ny/2), int(Nx/2-1):int(Nx/2+1)] = [-1, 1]
    dy[int(Ny/2-1):int(Ny/2+1), int(Nx/2)] = [-1, 1]
    F_denom = np.abs(fft2(dx))**2.0 + np.abs(fft2(dy))**2.0
    if D > 1: F_denom = np.dstack([F_denom]*D)

    S_history = [S]
    beta = lmd * 2.0
    hp, vp = np.zeros_like(S), np.zeros_like(S)
    for i in range(max_iter):
        # with S, solve for hp and vp in Eq. (12)
        hp, vp = circulant2_dx(S, 1), circulant2_dy(S, 1)
        if D == 1:
            mask = hp**2.0 + vp**2.0 < lmd/beta
        else:
            mask = np.sum(hp**2.0 + vp**2.0, axis=2) < lmd/beta
        hp[mask] = 0.0
        vp[mask] = 0.0

        # with hp and vp, solve for S in Eq. (8)
        hv = circulant2_dx(hp, -1) + circulant2_dy(vp, -1)
        S = np.real(ifft2((F_I + (beta*fft2(hv, axes=(0, 1))))/(1.0 + beta*F_denom), axes=(0, 1)))

        # iteration step
        if return_history:
            S_history.append(np.array(S))
        beta *= beta_rate
        if beta > beta_max: break

    if return_history:
        return S_history

    return S

In [6]:

# 2D
def circulantshift2_x(xs, h):
    return np.hstack([xs[:, h:], xs[:, :h]] if h > 0 else [xs[:, h:], xs[:, :h]])

def circulantshift2_y(xs, h):
    return np.vstack([xs[h:, :], xs[:h, :]] if h > 0 else [xs[h:, :], xs[:h, :]])

def circulant2_dx(xs, h):
    return (circulantshift2_x(xs, h) - xs)

def circulant2_dy(xs, h):
    return (circulantshift2_y(xs, h) - xs)

In [7]:
def l0_gradient(img,best_lmd=0.05):
    lmd=0.02
    sigma = 0.06
    beta_max=1.0e5
    beta_rate=2.0
    img_noise = add_noise(img, sigma)
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))
    fig.suptitle((r'$L_0$ Gradient Minimization on 2D Image (noise $\sigma={:.3}$).' + '\n'
        + r'$\beta_{{max}}={:.2e}, \kappa={:.3f}$').format(sigma, beta_max, beta_rate),
        fontsize=16)
    axs[0, 0].imshow(img)
    axs[0, 0].set_title('original')
    axs[0, 1].imshow(img_noise)
    axs[0, 1].set_title('noise')

    denoise_axs = [axs[0, 2], axs[1, 0], axs[1, 1], axs[1, 2]]
    best_res = 0
    for ax, lmd in zip(denoise_axs, [0.002, 0.01, 0.03, best_lmd]):
        print(f"ax:{ax},lmd:{lmd}")
        res = l0_gradient_minimization_2d(img_noise, lmd, beta_max, beta_rate)
        if lmd==best_lmd:
            best_res = res
        ax.imshow(clip_img(res), interpolation='nearest')
        ax.set_title('denoise $\\lambda = %f$' % lmd)
    return clip_img(best_res)


In [8]:
def line_reduce(lines):
    i = 0
    j = 0
    lines_final = []
    while i < len(lines) - 1:
        if j >= len(lines) - 1:
            break
        j = i + 1
        lines_final.append(lines[i])
        while j < len(lines) - 1:
            if lines[j][0] - lines[i][0] > 10:
                i = j
                break
            else:
                j = j + 1
    return lines_final

def line_sifting(lines_list):
    lines = []
    for rho, theta in lines_list[:]:
        if (theta < (np.pi / 6.0)) or (theta > (11 * np.pi / 6.0)) or ((theta > (5 *np.pi / 6.0)) and (theta < (7 * np.pi / 6.0))):  # 限制与y轴夹角小于30度的线
            lines.append([rho, theta])
    lines.sort()
    lines_final = line_reduce(lines)
    return lines_final

def draw(img, lines):
    new_img = img.copy()
    for rho, theta in lines[:]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * a)
        cv2.line(new_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return new_img

def draw_vertical(img, lines,img_width):
    new_img = img.copy()
    for rho, theta in lines[:]:
        x1 = int(rho)
        x2 = int(rho)
        y1 = 0
        y2 = img_width
        print(len(new_img),x1,x2,y1,y2)
        cv2.line(new_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return new_img

In [9]:
import cv2
import numpy as np

width =600
height = 450
file_name='1.jpg'

img = cv2.imread(file_name,0)
img_resized = cv2.resize(img,(width,height))
img_morph = morph_recons(img_resized,width=width,height=height)
img_l0 = l0_gradient(img_morph,best_lmd=0.2)
img_blur = cv2.GaussianBlur(img_morph,(3,3),0)
cv2.imshow("test", img_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

edges = cv2.Canny(img_blur,50,150,apertureSize = 3)

length_max =160
lines = cv2.HoughLines(edges,1,np.pi/180,length_max)
lines1 = lines[:,0,:]
houghlines = line_sifting(lines1)

new_img = draw_vertical(img_resized,houghlines,img_width=width)
cv2.imshow("test", new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ax:AxesSubplot(0.672059,0.536818;0.227941x0.343182),lmd:0.002
225.0, 299.0,301.0
ax:AxesSubplot(0.125,0.125;0.227941x0.343182),lmd:0.01
225.0, 299.0,301.0
ax:AxesSubplot(0.398529,0.125;0.227941x0.343182),lmd:0.03
225.0, 299.0,301.0
ax:AxesSubplot(0.672059,0.125;0.227941x0.343182),lmd:0.2
225.0, 299.0,301.0
450 -1 -1 0 600
450 158 158 0 600
450 191 191 0 600
450 239 239 0 600
450 269 269 0 600
450 324 324 0 600
450 336 336 0 600
450 358 358 0 600
450 373 373 0 600
450 386 386 0 600
450 408 408 0 600
